In [10]:
from jupyter_dash import JupyterDash

In [431]:
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State, ClientsideFunction
import dash_table
import dash_table.FormatTemplate as FormatTemplate

In [432]:
# Import all necessary libraries
import math # Needed for basic math operations\n",
import pandas as pd # Needed fpr dataframe creation and operations\n",
import numpy as np # Needed for array manipulations\n",
from itertools import islice # Needed for more complex row and coloumn slicing\n",
import matplotlib.pyplot as plt # Needed for quickly ploting results"
import pathlib
import plotly.express as px
import plotly.graph_objects as go

In [433]:
FONT_AWESOME = "https://use.fontawesome.com/releases/v5.10.2/css/all.css"

In [435]:
JupyterDash.infer_jupyter_proxy_config()

In [436]:
external_stylesheets = [dbc.themes.SPACELAB, FONT_AWESOME]
app = JupyterDash(__name__,external_stylesheets=external_stylesheets)
server = app.server
#app.layout = html.Div([
    
#    html.H1("Olympus Playground"),
#    html.Label('Initial Ohms'),
#    dcc.Input(id='initialOhms',type='number',placeholder='1.0',value=1,min=0.01, max=10000, step=0.01),
#    html.Label('Current reward yield'),
#    dcc.Input(id='rewardYield',type='number',placeholder='0.30',value=0.3058,min=0.01, max=0.4583, step=0.01),
    
#    dcc.Graph(id='ohmGrowthChart'),
    
#    html.Label('Incooom based on current Reward Yield'),
#    dcc.Input(id='desiredUSDTarget',type='number',placeholder='50000',value=50000,min=1, max=5000000, step=1),
#    dcc.Input(id='desiredOHMTarget',type='number',placeholder='1000',value=1000,min=0.01, max=100000, step=0.01),
#    dcc.Input(id='desiredDailyIncooom',type='number',placeholder='2500',value=2500,min=10, max=100000, step=0.01),
#    dcc.Input(id='desiredWeeklyIncooom',type='number',placeholder='5000',value=5000,min=1000, max=100000, step=0.01),
    
#    html.Label('ROI breakdown'),
#    html.Table([
#        html.Tr([html.Td(['Daily ROI: ']), html.Td(id='dailyROI_P')]),
#        html.Tr([html.Td(['5 Day ROI: ']), html.Td(id='fivedayROI_P')]),
#        html.Tr([html.Td(['7 Day ROI: ']), html.Td(id='sevendayROI_P')]),
#        html.Tr([html.Td(['Monthly ROI: ']), html.Td(id='monthlyROI_P')]),
#        html.Tr([html.Td(['Annual ROI: ']), html.Td(id='annualROI_P')]),
#    ]), 
    
#    html.Label('Incooom forcasts'),
#    html.Table([
#        html.Tr([html.Td(['Number of days until you reach target USD from staking only: ']), html.Td(id='forcastUSDTarget')]),
#        html.Tr([html.Td(['Number of days until you reach target OHM from staking only:']), html.Td(id='forcastOHMTarget')]),
#        html.Tr([html.Td(['Number of days until you reach your target daily incooom:']), html.Td(id='forcastDailyIncooom')]),
#        html.Tr([html.Td(['Required OHMs to achieve your target daily incooom:']), html.Td(id='requiredOHMDailyIncooom')]),
#        html.Tr([html.Td(['Number of days until you reach your target weekly incooom:']), html.Td(id='forcastWeeklyIncooom')]),
#        html.Tr([html.Td(['Required OHMs to achieve your target weekly incooom:']), html.Td(id='requiredOHMWeeklyIncooom')]),
#    ]),
#])

# App Layout: Bootstrap
#====================================================================
 
#====================================================================
app.layout = dbc.Container([
    # Title row
    dbc.Row([
        dbc.Col(html.H1("OlympusDAO Playground",
                        className = 'text-center text-primary, mb-4'),
                width = 12 )
    ]),
#====================================================================
    
#====================================================================
# Instruction and ohm growth chart row
    dbc.Row([
        # Welcome message and Instructions column
        dbc.Col([ 
            dbc.Card(
                dbc.CardBody([
                        dcc.Markdown(''' 
                        #### Welcome to Olympus playground
                        Playground is a simulator for staking, bonding and incooom strategies.
                        
                        Use this simulator to:
                        - Forcast ROI ar current and future reward yield percent
                        - OHM growth over time
                        - OHM and USD value over time
                        
                        We love the incooom, use this simulator to strategize:
                        - Required staked OHM to reach desired daily incooom
                        - Count down until you are earning desired daily incooom
                        - Required staked OHM to reach desired weekly incooom
                        - Count down until you are earning desired weekly incooom
                        '''),
                        
                    ]))], width = 3),
        
        # ohm growth output column    
        dbc.Col([
            dcc.Graph(id = 'ohmGrowthChart', figure = {'layout':{'title': 'Ohm Growth Chart vs Days'}},)
            
        ])
    ], align = 'center'),
#====================================================================

#====================================================================
# Input form, ROI and incooom row 
    dbc.Row([
        # Input form. The input form should be embedded inside a bootsrap card
        dbc.Col([
            dbc.Card(
            dbc.CardBody([
                dcc.Markdown('''
                #### Input form
                '''),
            ]))], width = 3),
        
        dbc.Col([
            dbc.Card(
            dbc.CardBody([
                dcc.Markdown('''
                #### ROI Outputs
                '''),
            ]))], width = 3),
        
        dbc.Col([
            dbc.Card(
            dbc.CardBody([
                dcc.Markdown('''
                #### Incooom forcast outputs
                '''),
            ]))], width = 6),
        
    ], align = 'center')
#====================================================================    
  
], fluid = True)

In [437]:
# Pull most recent ohm price from coingecko
from pycoingecko import CoinGeckoAPI # Pulls live data from coingecko
cg = CoinGeckoAPI() # Pulls live price feed from coingecko
ohmPrice = cg.get_price(ids='olympus',vs_currencies='usd') # Get most recent price from coingecko via api
ohmPrice = ohmPrice['olympus']['usd']

In [438]:
# Data frame to hold all required data point. Data required would be Epochs since rebase are distributed every Epoch
ohmGrowth_df = pd.DataFrame(np.arange(3001), columns = ['Epochs']) # In this case let's consider 3000 Epochs which is 1000 days
ohmGrowth_df['Days'] = ohmGrowth_df.Epochs/3 # There are 3 Epochs per day so divide by 3 to get Days
#================================================================================
roiTablulated_df = []
#================================================================================

In [439]:
# Ohm growth section: This section calculates and plots the projected ohm growth over time (1000 days)

In [440]:
# Define callback to update the ohmGrowthChart. As user changes the initial staked ohm, the chart updates dynamically 
@app.callback(
    Output('ohmGrowthChart', 'figure'),
    Input("initialOhms", "value"),
    Input('rewardYield','value')
)
#================================================================================

# This function calculates and generates the ohm growth over time using the initial staked ohm as a starting point
def update_figure(initialOhms,rewardYield):
    
# To Calculate the ohm growth over 3000 Epochs or 1000 days, we loop through the exponential ohm growth equation every epoch

    totalOhms = [] # create an empty array that will hold the componded rewards
    rewardYield = round(rewardYield/100,5)
    
    ohmStakedGrowth = initialOhms # Initial staked ohms used to project growth over time
    for elements in ohmGrowth_df.Epochs: # Initialize the for loop to have loops equal to number of rows or number of epochs
        totalOhms.append(ohmStakedGrowth) # populate the empty array with calclated values each iteration
        ohmStakedGrowth = ohmStakedGrowth*(1+rewardYield) # compound the total amount of ohms 
    ohmGrowth_df['Total_Ohms'] = totalOhms # Clean up and add the new array to the main data frame
    ohmGrowth_df.Days = np.around(ohmGrowth_df.Days,decimals=1) # Python is funny so let's round up our numbers . 1 decimal place for days",
    ohmGrowth_df.Total_Ohms = np.around(ohmGrowth_df.Total_Ohms,decimals=3)  # Python is funny so let's round up our numbers . 3 decimal place for ohms"

#================================================================================  
    
    return px.line(
        ohmGrowth_df, x="Days", y="Total_Ohms",
        render_mode="webgl", title="Accumulated Ohms"
    )

In [441]:
# ROI Section: This section 

In [442]:
# Define callback to update the ohmGrowthChart. As user changes the initial staked ohm, the chart updates dynamically 
@app.callback(
    Output('dailyROI_P', 'children'),
    Output('fivedayROI_P', 'children'),
    Output('sevendayROI_P', 'children'),
    Output('monthlyROI_P', 'children'),
    Output('annualROI_P', 'children'),
    Output('forcastUSDTarget', 'children'),
    Output('forcastOHMTarget', 'children'),
    Output('forcastDailyIncooom', 'children'),
    Output('requiredOHMDailyIncooom', 'children'),
    Output('forcastWeeklyIncooom', 'children'),
    Output('requiredOHMWeeklyIncooom', 'children'),
    Input("rewardYield", "value"),
    Input("initialOhms", "value"),
    Input("desiredUSDTarget", "value"),
    Input("desiredOHMTarget", "value"),
    Input("desiredDailyIncooom", "value"),
    Input("desiredWeeklyIncooom", "value")
)
#================================================================================
def update_Incooom(rewardYield,initialOhms,desiredUSDTarget,desiredOHMTarget,desiredDailyIncooom,desiredWeeklyIncooom):
    ohmStakedInit = initialOhms
    rewardYield = round(rewardYield/100,5)
    rebaseConst = 1 + rewardYield
    currentAPY = 17407/100 # current staking %APY. Need to make this read from a source or user entry
    ohmPrice = 300
    
    # Let's get some ROI Outputs starting with the daily
    dailyROI = (1+rewardYield)**3-1 # Equation to calculate your daily ROI based on reward Yield
    dailyROI_P = round(dailyROI*100,2) # daily ROI in Percentage
    #================================================================================

    # 5 day ROI
    fivedayROI = (1+rewardYield)**(5*3)-1 # Equation to calculate your 5 day ROI based on reward Yield
    fivedayROI_P = round(fivedayROI*100,2) # 5 day ROI in Percentage
    #================================================================================

    # 7 day ROI
    sevendayROI = (1+rewardYield)**(7*3)-1 # Equation to calculate your 7 day ROI based on reward Yield
    sevendayROI_P = round(sevendayROI*100,2) # 7 day ROI in Percentage
    #================================================================================

    # 30 day ROI
    monthlyROI = (1+rewardYield)**(30*3)-1 # Equation to calculate your 30 day ROI based on reward Yield
    monthlyROI_P = round(monthlyROI*100,2) # 30 day ROI in Percentage
    #================================================================================

    # Annual ROI
    annualROI = (1+rewardYield)**(365*3)-1 # Equation to calculate your annual ROI based on reward Yield
    annualROI_P = round(annualROI*100,2) # Equation to calculate your annual ROI based on reward Yield
    #================================================================================

    # Let's create a nice looking table to view the results of our calculations. The table will contain the ROIs and the percentages
    roiData = [['Daily',dailyROI_P],['5 Day',fivedayROI_P],['7 Day',sevendayROI_P],['1 Month',monthlyROI_P],['1 Year',annualROI_P]]
    roiTablulated_df = pd. DataFrame(roiData,columns = ['Cadence','Percentage'])
    #================================================================================
    
    # Days until you reach target USD by staking only
    forcastUSDTarget = round((math.log(desiredUSDTarget/(ohmStakedInit*ohmPrice),rebaseConst)/3))
    #================================================================================

    # Days until you reach target OHM by staking only
    forcastOHMTarget = round(math.log(desiredOHMTarget/(ohmStakedInit),rebaseConst)/3) 
    #================================================================================

    # Daily Incooom calculations
    # Required OHMs until you are earning your desired daily incooom
    requiredOHMDailyIncooom = round((desiredDailyIncooom/dailyROI)/ohmPrice)
    # Days until you are earning your desired daily incooom from your current initial staked OHM amount
    forcastDailyIncooom = round(math.log((requiredOHMDailyIncooom/ohmStakedInit),rebaseConst)/3)
    requiredUSDForDailyIncooom = requiredOHMDailyIncooom*ohmPrice
    #================================================================================

    # Weekly Incooom calculations
    # Required OHMs until you are earning your desired weekly incooom
    requiredOHMWeeklyIncooom = round((desiredWeeklyIncooom/sevendayROI)/ohmPrice)
    # Days until you are earning your desired weekly incooom from your current initial staked OHM amount
    forcastWeeklyIncooom = round(math.log((requiredOHMWeeklyIncooom/ohmStakedInit),rebaseConst)/3)
    requiredUSDForWeeklyIncooom = requiredOHMWeeklyIncooom*ohmPrice
    #================================================================================
    
    return dailyROI_P,fivedayROI_P,sevendayROI_P,monthlyROI_P,annualROI_P,forcastUSDTarget,forcastOHMTarget,forcastDailyIncooom,requiredOHMDailyIncooom,forcastWeeklyIncooom,requiredOHMWeeklyIncooom  
  

In [443]:
# Run app and display result inline in the notebook
app.run_server()

Dash app running on http://127.0.0.1:8050/


In [1]:
%watermarks iversions

UsageError: Line magic function `%watermarks` not found.


In [8]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [9]:
%watermark --iversions